In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
import matplotlib.pyplot as plt


In [2]:
train_path = "preprocessed dataset/training"
test_path = "preprocessed dataset/testing"


In [3]:
IMAGE_SIZE = (299, 299)
BATCH_SIZE = 32
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    brightness_range=[0.8,1.2],
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


In [4]:
training_set = train_datagen.flow_from_directory(
    train_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


In [5]:
base_model = Xception(
    input_shape=IMAGE_SIZE + (3,),
    weights='imagenet',
    include_top=False
)


In [6]:
for layer in base_model.layers:
    layer.trainable = False


In [7]:
x = Flatten()(base_model.output)

x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

output = Dense(5, activation='softmax')(x)   # 5 DR classes

model = Model(inputs=base_model.input, outputs=output)


In [8]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [9]:
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 299, 299, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1 (Conv2D)         │ (None, 149, 149, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1_bn               │ (None, 149, 149, 32)      │             128 │ block1_conv1[0][0]         │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1_act (Activation) │ (None, 149, 149, 32)      │               0 │ block1_conv1_bn[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2 (Conv2D)         │ (None, 147, 147, 64)      │          18,432 │ block1_conv1_act[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2_bn               │ (None, 147, 147, 64)      │             256 │ block1_conv2[0][0]         │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2_act (Activation) │ (None, 147, 147, 64)      │               0 │ block1_conv2_bn[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv1               │ (None, 147, 147, 128)     │           8,768 │ block1_conv2_act[0][0]     │
│ (SeparableConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv1_bn            │ (None, 147, 147, 128)     │             512 │ block2_sepconv1[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2_act           │ (None, 147, 147, 128)     │               0 │ block2_sepconv1_bn[0][0]   │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2               │ (None, 147, 147, 128)     │          17,536 │ block2_sepconv2_act[0][0]  │
│ (SeparableConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2_bn            │ (None, 147, 147, 128)     │             512 │ block2_sepconv2[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 74, 74, 128)       │           8,192 │ block1_conv2_act[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_pool (MaxPooling2D)    │ (None, 74, 74, 128)       │               

 Total params: 73,291,821 (279.59 MB)

 Trainable params: 52,430,341 (200.01 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [10]:
# Early Stopping Callback
early_stop = EarlyStopping(
    monitor='val_loss',     # Monitor validation loss
    patience=5,             # Stop after 5 epochs without improvement
    restore_best_weights=True,  # Restore best model weights
    verbose=1
)


In [11]:
checkpoint = ModelCheckpoint(
    'best_xception_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)


In [12]:
EPOCHS = 30

history = model.fit(
    training_set,
    steps_per_epoch=training_set.samples // BATCH_SIZE,
    validation_data=test_set,
    validation_steps=test_set.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stop, checkpoint]   
)


Epoch 1/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5412 - loss: 9.7614
Epoch 1: val_loss improved from None to 1.02110, saving model to best_xception_model.h5


114/114 ━━━━━━━━━━━━━━━━━━━━ 575s 5s/step - accuracy: 0.6019 - loss: 4.2061 - val_accuracy: 0.5781 - val_loss: 1.0211
Epoch 2/30
  1/114 ━━━━━━━━━━━━━━━━━━━━ 5:43 3s/step - accuracy: 0.6562 - loss: 0.9811

C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_loss did not improve from 1.02110
114/114 ━━━━━━━━━━━━━━━━━━━━ 77s 653ms/step - accuracy: 0.6562 - loss: 0.9811 - val_accuracy: 0.5881 - val_loss: 1.0238
Epoch 3/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6633 - loss: 1.0537
Epoch 3: val_loss improved from 1.02110 to 0.90296, saving model to best_xception_model.h5


114/114 ━━━━━━━━━━━━━━━━━━━━ 558s 5s/step - accuracy: 0.6716 - loss: 1.0183 - val_accuracy: 0.7003 - val_loss: 0.9030
Epoch 4/30
  1/114 ━━━━━━━━━━━━━━━━━━━━ 5:33 3s/step - accuracy: 0.6562 - loss: 1.1668
Epoch 4: val_loss improved from 0.90296 to 0.89584, saving model to best_xception_model.h5


114/114 ━━━━━━━━━━━━━━━━━━━━ 77s 658ms/step - accuracy: 0.6562 - loss: 1.1668 - val_accuracy: 0.6989 - val_loss: 0.8958
Epoch 5/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6712 - loss: 1.0037
Epoch 5: val_loss did not improve from 0.89584
114/114 ━━━━━━━━━━━━━━━━━━━━ 557s 5s/step - accuracy: 0.6722 - loss: 0.9818 - val_accuracy: 0.6790 - val_loss: 0.9158
Epoch 6/30
  1/114 ━━━━━━━━━━━━━━━━━━━━ 5:39 3s/step - accuracy: 0.6562 - loss: 0.9783
Epoch 6: val_loss did not improve from 0.89584
114/114 ━━━━━━━━━━━━━━━━━━━━ 76s 649ms/step - accuracy: 0.6562 - loss: 0.9783 - val_accuracy: 0.6619 - val_loss: 0.9633
Epoch 7/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6858 - loss: 0.9682
Epoch 7: val_loss improved from 0.89584 to 0.83250, saving model to best_xception_model.h5


114/114 ━━━━━━━━━━━━━━━━━━━━ 555s 5s/step - accuracy: 0.6890 - loss: 0.9442 - val_accuracy: 0.7216 - val_loss: 0.8325
Epoch 8/30
  1/114 ━━━━━━━━━━━━━━━━━━━━ 5:25 3s/step - accuracy: 0.8750 - loss: 0.6519
Epoch 8: val_loss did not improve from 0.83250
114/114 ━━━━━━━━━━━━━━━━━━━━ 75s 638ms/step - accuracy: 0.8750 - loss: 0.6519 - val_accuracy: 0.7202 - val_loss: 0.8387
Epoch 9/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6785 - loss: 0.9314
Epoch 9: val_loss improved from 0.83250 to 0.79515, saving model to best_xception_model.h5


114/114 ━━━━━━━━━━━━━━━━━━━━ 552s 5s/step - accuracy: 0.6860 - loss: 0.9038 - val_accuracy: 0.7131 - val_loss: 0.7951
Epoch 10/30
  1/114 ━━━━━━━━━━━━━━━━━━━━ 5:26 3s/step - accuracy: 0.6875 - loss: 0.7756
Epoch 10: val_loss did not improve from 0.79515
114/114 ━━━━━━━━━━━━━━━━━━━━ 75s 639ms/step - accuracy: 0.6875 - loss: 0.7756 - val_accuracy: 0.7102 - val_loss: 0.7955
Epoch 11/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6954 - loss: 0.8813
Epoch 11: val_loss did not improve from 0.79515
114/114 ━━━━━━━━━━━━━━━━━━━━ 551s 5s/step - accuracy: 0.7030 - loss: 0.8610 - val_accuracy: 0.6946 - val_loss: 0.8638
Epoch 12/30
  1/114 ━━━━━━━━━━━━━━━━━━━━ 5:32 3s/step - accuracy: 0.6875 - loss: 1.1665
Epoch 12: val_loss did not improve from 0.79515
114/114 ━━━━━━━━━━━━━━━━━━━━ 75s 640ms/step - accuracy: 0.6875 - loss: 1.1665 - val_accuracy: 0.6918 - val_loss: 0.8669
Epoch 13/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7054 - loss: 0.8379
Epoch 13: val_loss improved from

114/114 ━━━━━━━━━━━━━━━━━━━━ 551s 5s/step - accuracy: 0.7105 - loss: 0.8280 - val_accuracy: 0.7358 - val_loss: 0.7379
Epoch 14/30
  1/114 ━━━━━━━━━━━━━━━━━━━━ 5:26 3s/step - accuracy: 0.7500 - loss: 0.7502
Epoch 14: val_loss improved from 0.73794 to 0.73374, saving model to best_xception_model.h5


114/114 ━━━━━━━━━━━━━━━━━━━━ 76s 650ms/step - accuracy: 0.7500 - loss: 0.7502 - val_accuracy: 0.7358 - val_loss: 0.7337
Epoch 15/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7042 - loss: 0.8576
Epoch 15: val_loss did not improve from 0.73374
114/114 ━━━━━━━━━━━━━━━━━━━━ 550s 5s/step - accuracy: 0.7000 - loss: 0.8538 - val_accuracy: 0.6804 - val_loss: 0.9515
Epoch 16/30
  1/114 ━━━━━━━━━━━━━━━━━━━━ 5:31 3s/step - accuracy: 0.8750 - loss: 0.5798
Epoch 16: val_loss did not improve from 0.73374
114/114 ━━━━━━━━━━━━━━━━━━━━ 75s 640ms/step - accuracy: 0.8750 - loss: 0.5798 - val_accuracy: 0.6804 - val_loss: 0.9759
Epoch 17/30
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7043 - loss: 0.8763
Epoch 17: val_loss did not improve from 0.73374
114/114 ━━━━━━━━━━━━━━━━━━━━ 549s 5s/step - accuracy: 0.7096 - loss: 0.8379 - val_accuracy: 0.7060 - val_loss: 0.7848
Epoch 18/30
  1/114 ━━━━━━━━━━━━━━━━━━━━ 5:32 3s/step - accuracy: 0.8750 - loss: 0.5991
Epoch 18: val_loss did not impro

In [13]:
model.save("models/Updated-Xception-diabetic-retinopathy.h5")

print("✅ Model Saved Successfully")


✅ Model Saved Successfully
